In [8]:
import pandas as pd
import json

In [9]:
data = {"resume": [], "label": []}
df = pd.DataFrame(data)

# Iterate through a folder of text files and extract the text
import os
import codecs

# Define the directory
dir_path = "./resumes_corpus"

# Get a list of all files in the directory
files = os.listdir(dir_path)

# Iterate over each file
for file in files:
    if file.endswith(".txt"):
        # Construct full file path
        file_path = os.path.join(dir_path, file)
        file_path_label = file_path.replace(".txt", ".lab")
        data = {"resume": [], "label": []}

        # Open the file
        with codecs.open(file_path, "r", encoding="utf8", errors="ignore") as f:
            # Read the file's contents
            content = f.read()
            data["resume"].append(content)

        # Open the file
        with codecs.open(file_path_label, "r", encoding="utf8", errors="ignore") as f:
            # Read the file's contents
            content = f.read()
            content = content.split("\n")
            if "" in content and len(content) == 1:
                continue
            elif "" in content:
                content.remove("")
            data["label"].append(content)

        new_row_df = pd.DataFrame(data)
        # Add the new row to the DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)

In [10]:
unique_labels = df["label"].explode().unique()
df_generator = df.iterrows()
valid_data = []
indices = []
for i in range(10):
    for u_label in unique_labels:
        data = {u_label: {"pos": [], "neg": []}}
        i = 0
        while i < 5:
            index, row = next(df_generator)
            if u_label in row["label"]:
                data[u_label]["pos"].append(row["resume"])
                indices.append(index)
                i += 1
        i = 0
        while i < 15:
            index, row = next(df_generator)
            if u_label not in row["label"]:
                indices.append(index)
                data[u_label]["neg"].append(row["resume"])
                i += 1
        valid_data.append(data)

In [11]:
test_data = []
for i in range(10):
    for u_label in unique_labels:
        data = {u_label: {"pos": [], "neg": []}}
        i = 0
        while i < 5:
            index, row = next(df_generator)
            if u_label in row["label"]:
                data[u_label]["pos"].append(row["resume"])
                indices.append(index)
                i += 1
        i = 0
        while i < 15:
            index, row = next(df_generator)
            if u_label not in row["label"]:
                indices.append(index)
                data[u_label]["neg"].append(row["resume"])
                i += 1
        test_data.append(data)

In [12]:
train_data = df.drop(indices)

In [13]:
train_data.to_json("train_data.json", orient="records", indent=4)
with open("valid_data.json", "w") as f:
    json.dump(valid_data, f, indent=4)
with open("test_data.json", "w") as f:
    json.dump(test_data, f, indent=4)